In [19]:
def calculate_limb(distance_matrix, n, j):
    return min(distance_matrix[i][j] + distance_matrix[k][j] - distance_matrix[i][k]
               for i in range(n) for k in range(n) if i != j and k != j) // 2

def remove_edge(graph, src, dst):
    for i, (neighbor, length) in enumerate(graph[src]):
        if neighbor == dst:
            break
    del graph[src][i]

    for i, (neighbor, length) in enumerate(graph[dst]):
        if neighbor == src:
            break
    del graph[dst][i]

def find_path(graph, src, dst, visited):
    visited[src] = True
    for v, w in graph[src]:
        if visited[v]:
            continue

        if v == dst:
            return [(src, w), (dst, 0)]

        path = find_path(graph, v, dst, visited)
        if path is not None:
            return [(src, w)] + path

    return None

def insert_node(graph, src, dst, distance, visited, node_name):

    path = find_path(graph, src, dst, visited)

    curr = 0 
    edge_length = path[0][1]
    remaining_length = distance


    while remaining_length >= edge_length:
        remaining_length -= edge_length
        curr += 1
        edge_length = path[curr][1]

    current_node = path[curr][0]
    next_node = path[curr + 1][0]


    graph[current_node].append((node_name, remaining_length))
    graph[next_node].append((node_name, edge_length - remaining_length))
    graph[node_name] = [(current_node, remaining_length), (next_node, edge_length - remaining_length)]
    remove_edge(graph, current_node, next_node)


def phylogeny(distance_matrix, n, node_name):

    if n == 2:
        distance = distance_matrix[0][1]
        return {0: [(1, distance)], 1: [(0, distance)]}


    limb_length = calculate_limb(distance_matrix, n=n, j=n-1)
    for i in range(n-1):
        distance_matrix[i][n-1] -= limb_length
        distance_matrix[n-1][i] = distance_matrix[i][n-1]


    for i in range(n-1):
        for j in range(i + 1, n-1):
            if distance_matrix[i][j] == distance_matrix[i][n-1] + distance_matrix[j][n-1]:
                x = distance_matrix[i][n-1]
                src, dst = i, j  # store i, j in src, dst
                break


    graph = phylogeny(distance_matrix, n - 1, node_name=node_name - 1)


    visited = [False] * (2 * len(distance_matrix))
    insert_node(graph, src, dst, x, visited, node_name)

    graph[n - 1] = [(node_name, limb_length)]
    graph[node_name].append((n - 1, limb_length))

    return graph


In [20]:
with open("/Users/shayanaryania/Desktop/University/Rosalind/Implement_AdditivePhylogeny/rosalind_ba7c.txt") as File:
    N = int(File.readline())
    Matrix = [list(map(int, line.split())) for line in File.readlines()]
    graph = phylogeny(Matrix, N, node_name=2 * len(Matrix) - 3)

In [21]:
with open("/Users/shayanaryania/Desktop/University/Rosalind/Implement_AdditivePhylogeny/rosalind_BA7C_out.txt", "w") as out:
    for current in graph:
        for next, length in graph[current]:
            print('%d->%d:%d' % (current, next, length), file=out)